# 01 Build Daily Panel

Aggregate player history to a daily panel and join price data.

In [ ]:
from pathlib import Path
import sys

def find_repo_root():
    cwd = Path().resolve()
    for candidate in (cwd,) + tuple(cwd.parents):
        if (candidate / 'src').is_dir() and (candidate / 'config.yaml').exists():
            return candidate
    return cwd

ROOT = find_repo_root()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.panel import build_daily_panel, write_ingestion_audit

raw_dir = ROOT / 'data/raw/mendeley'
panel, meta = build_daily_panel(str(raw_dir))

audit = write_ingestion_audit(raw_dir, meta)
print('Wrote ingestion audit to reports/ingestion_audit.json and reports/ingestion_audit.md')

out_path = ROOT / 'data/processed/panel_daily.parquet'
out_path.parent.mkdir(parents=True, exist_ok=True)
panel.to_parquet(out_path, index=False)

print('Rows:', len(panel))
display(panel.head())
display(panel.describe())

if meta.get('player_errors'):
    print('Player file errors:')
    for err in meta['player_errors']:
        print(f'- {err}')
if meta.get('price_errors'):
    print('Price file errors:')
    for err in meta['price_errors']:
        print(f'- {err}')
